In [1]:
from ray.rllib.algorithms.ppo import PPO
from ray import air, tune
import ray

from esc.epanet_env import N_SIMULATION_STEPS, EPANETEnv

/opt/homebrew/Caskroom/mambaforge/base/envs/esc-m1/lib/python3.10/site-packages/tensorflow/core/framework/tensor_shape_pb2.py:18: DeprecationWarning: Call to deprecated create function FileDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
  DESCRIPTOR = _descriptor.FileDescriptor(
/opt/homebrew/Caskroom/mambaforge/base/envs/esc-m1/lib/python3.10/site-packages/tensorflow/core/framework/tensor_shape_pb2.py:36: DeprecationWarning: Call to deprecated create function FieldDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
  _descriptor.FieldDescriptor(
/opt/homebrew/Caskroom/mambaforge/base/envs/esc-m1/lib/python3.10/site-packages/tensorflow/core/framework/tensor_shape_pb2.py:29: DeprecationWarning: Call to deprecated create function Descriptor(). Note: Create unlinked descriptors is going to 

In [2]:
ray.init(num_cpus=10, num_gpus=1, runtime_env={"env_vars": { "CUDA_VISIBLE_DEVICES": "" }})

2022-12-02 00:16:52,848	INFO worker.py:1528 -- Started a local Ray instance.


TypeError: runtime_env['env_vars'] must be of type Dict[str, str]

In [ ]:

N_TRAINING_ITERATIONS = 1000
gpu_count = 1.0
driver_gpu = 1.0
num_workers = 9
num_gpus_per_worker = (gpu_count - driver_gpu) / num_workers
# Configure the algorithm.
config = {
    "env_config": {},
    # Use 5 environment workers (aka "rollout workers") that parallelly
    # collect samples from their own environment clone(s).
    "num_workers": num_workers,
    "create_env_on_driver": False,
    "num_gpus": driver_gpu,
    "num_gpus_per_worker": num_gpus_per_worker,
    "horizon": N_SIMULATION_STEPS,
    "train_batch_size": N_SIMULATION_STEPS * num_workers,
    "rollout_fragment_length": N_SIMULATION_STEPS,
    "batch_mode": "complete_episodes",
    # Change this to "framework: torch", if you are using PyTorch.
    # Also, use "framework: tf2" for tf2.x eager execution.
    "framework": "tf2",
    "eager_tracing": True,
    # Tweak the default model provided automatically by RLlib,
    # given the environment's observation- and action spaces.
    "model": {
        "fcnet_hiddens": [256, 256],
        "fcnet_activation": "relu",
        "use_lstm": True,
    },
    "recreate_failed_workers": True,
    "log_sys_usage": False,
    # Set up a separate evaluation worker set for the
    # `algo.evaluate()` call after training (see below).
    "evaluation_num_workers": 1,
    # Only for evaluation runs, render the env.
    # "evaluation_config": {
    #     "render_env": True,
    # },
    "log_level": "ERROR"
}

# Create our RLlib Trainer.
algo = PPO(env=EPANETEnv, config=config)
# algo = PPO.from_checkpoint("/Users/maxdumas/ray_results/PPO_EPANETEnv_2022-11-27_22-44-29tztzkh8w/checkpoint_000451")

# Run it for n training iterations. A training iteration includes
# parallel sample collection by the environment workers as well as
# loss calculation on the collected batch and a model update.
for i in range(N_TRAINING_ITERATIONS):
    print(algo.train())

    if i % 50 == 0:
        checkpoint = algo.save()
        print(f"Checkpoint saved to {checkpoint}")

# Evaluate the trained Trainer (and render each timestep to the shell's
# output).
algo.evaluate()

# ray.init(num_cpus=10, num_gpus=1)

# tuner = tune.Tuner(tune.with_resources(APPO, {"cpu": 10, "gpu": 1}), param_space=config, run_config=air.RunConfig(
#     checkpoint_config=air.CheckpointConfig(checkpoint_frequency=10),
#     stop={
#         "training_iteration": 200
#     }))

# results = tuner.fit()
# results

2022-12-02 00:07:54,612	INFO algorithm.py:2292 -- Executing eagerly (framework='tf2'), with eager_tracing=True. For production workloads, make sure to set eager_tracing=True  in order to match the speed of tf-static-graph (framework='tf'). For debugging purposes, `eager_tracing=False` is the best choice.
2022-12-02 00:07:54,614	WARNING algorithm.py:2531 -- You have specified 1 evaluation workers, but your `evaluation_interval` is None! Therefore, evaluation will not occur automatically with each call to `Algorithm.train()`. Instead, you will have to call `Algorithm.evaluate()` manually in order to trigger an evaluation run.
2022-12-02 00:07:54,617	INFO algorithm.py:457 -- Current log_level is ERROR. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=18943) /opt/homebrew/Caskroom/mambaforge/base/envs/esc-m1/lib/python3.10/site-packages/tensorflow/core/framework/tensor_shape_pb2.py:18: DeprecationWarning: Call to deprecated create function FileDescr

(RolloutWorker pid=18939) EPANET version 20200 loaded (EPyT version 1.0.1).
(RolloutWorker pid=18939) Input File BUILDING.inp loaded successfully.
(RolloutWorker pid=18939) 
(RolloutWorker pid=18943) EPANET version 20200 loaded (EPyT version 1.0.1).
(RolloutWorker pid=18943) Input File BUILDING.inp loaded successfully.
(RolloutWorker pid=18943) 
(RolloutWorker pid=18942) EPANET version 20200 loaded (EPyT version 1.0.1).
(RolloutWorker pid=18942) Input File BUILDING.inp loaded successfully.
(RolloutWorker pid=18942) 
(RolloutWorker pid=18945) EPANET version 20200 loaded (EPyT version 1.0.1).
(RolloutWorker pid=18945) Input File BUILDING.inp loaded successfully.
(RolloutWorker pid=18945) 
(RolloutWorker pid=18946) EPANET version 20200 loaded (EPyT version 1.0.1).
(RolloutWorker pid=18946) Input File BUILDING.inp loaded successfully.
(RolloutWorker pid=18946) 
(RolloutWorker pid=18947) EPANET version 20200 loaded (EPyT version 1.0.1).
(RolloutWorker pid=18947) Input File BUILDING.inp load

(pid=19000) /opt/homebrew/Caskroom/mambaforge/base/envs/esc-m1/lib/python3.10/site-packages/tensorflow/core/framework/tensor_shape_pb2.py:18: DeprecationWarning: Call to deprecated create function FileDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
(pid=19000)   DESCRIPTOR = _descriptor.FileDescriptor(
(pid=19000) /opt/homebrew/Caskroom/mambaforge/base/envs/esc-m1/lib/python3.10/site-packages/tensorflow/core/framework/tensor_shape_pb2.py:36: DeprecationWarning: Call to deprecated create function FieldDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
(pid=19000)   _descriptor.FieldDescriptor(
(pid=19000) /opt/homebrew/Caskroom/mambaforge/base/envs/esc-m1/lib/python3.10/site-packages/tensorflow/core/framework/tensor_shape_pb2.py:29: DeprecationWarning: Call to deprecated create function 

(RolloutWorker pid=19000) EPANET version 20200 loaded (EPyT version 1.0.1).
(RolloutWorker pid=19000) Input File BUILDING.inp loaded successfully.
(RolloutWorker pid=19000) 


2022-12-02 00:08:04,683	INFO trainable.py:164 -- Trainable.setup took 10.074 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.


...

(RolloutWorker pid=18939) EPANET version 20200 loaded (EPyT version 1.0.1).
(RolloutWorker pid=18939) Input File BUILDING.inp loaded successfully.
(RolloutWorker pid=18939) 
(RolloutWorker pid=18943) EPANET version 20200 loaded (EPyT version 1.0.1).
(RolloutWorker pid=18943) Input File BUILDING.inp loaded successfully.
(RolloutWorker pid=18943) 
(RolloutWorker pid=18942) EPANET version 20200 loaded (EPyT version 1.0.1).
(RolloutWorker pid=18942) Input File BUILDING.inp loaded successfully.
(RolloutWorker pid=18942) 
(RolloutWorker pid=18945) EPANET version 20200 loaded (EPyT version 1.0.1).
(RolloutWorker pid=18945) Input File BUILDING.inp loaded successfully.
(RolloutWorker pid=18945) 
(RolloutWorker pid=18946) EPANET version 20200 loaded (EPyT version 1.0.1).
(RolloutWorker pid=18947) EPANET version 20200 loaded (EPyT version 1.0.1).
(RolloutWorker pid=18947) Input File BUILDING.inp loaded successfully.
(RolloutWorker pid=18947) 
(RolloutWorker pid=18944) EPANET version 20200 loaded 

(RolloutWorker pid=18940) /opt/homebrew/Caskroom/mambaforge/base/envs/esc-m1/lib/python3.10/site-packages/epyt/epanet.py:11948: UserWarning: WARNING: System has negative pressures.
(RolloutWorker pid=18940)   warnings.warn(errmssg.value.decode())
(RolloutWorker pid=18943) /opt/homebrew/Caskroom/mambaforge/base/envs/esc-m1/lib/python3.10/site-packages/epyt/epanet.py:11948: UserWarning: WARNING: System has negative pressures.
(RolloutWorker pid=18943)   warnings.warn(errmssg.value.decode())
(RolloutWorker pid=18939) /opt/homebrew/Caskroom/mambaforge/base/envs/esc-m1/lib/python3.10/site-packages/epyt/epanet.py:11948: UserWarning: WARNING: System has negative pressures.
(RolloutWorker pid=18939)   warnings.warn(errmssg.value.decode())
(RolloutWorker pid=18945) /opt/homebrew/Caskroom/mambaforge/base/envs/esc-m1/lib/python3.10/site-packages/epyt/epanet.py:11948: UserWarning: WARNING: System has negative pressures.
(RolloutWorker pid=18945)   warnings.warn(errmssg.value.decode())
(RolloutWork

(RolloutWorker pid=18939) EPANET version 20200 loaded (EPyT version 1.0.1).
(RolloutWorker pid=18939) Input File BUILDING.inp loaded successfully.
(RolloutWorker pid=18939) 
(RolloutWorker pid=18943) EPANET version 20200 loaded (EPyT version 1.0.1).
(RolloutWorker pid=18943) Input File BUILDING.inp loaded successfully.
(RolloutWorker pid=18943) 
(RolloutWorker pid=18945) EPANET version 20200 loaded (EPyT version 1.0.1).
(RolloutWorker pid=18945) Input File BUILDING.inp loaded successfully.
(RolloutWorker pid=18945) 
(RolloutWorker pid=18946) EPANET version 20200 loaded (EPyT version 1.0.1).
(RolloutWorker pid=18946) Input File BUILDING.inp loaded successfully.
(RolloutWorker pid=18946) 
(RolloutWorker pid=18944) EPANET version 20200 loaded (EPyT version 1.0.1).
(RolloutWorker pid=18944) Input File BUILDING.inp loaded successfully.
(RolloutWorker pid=18944) 
(RolloutWorker pid=18941) EPANET version 20200 loaded (EPyT version 1.0.1).
(RolloutWorker pid=18941) Input File BUILDING.inp load

/opt/homebrew/Caskroom/mambaforge/base/envs/esc-m1/lib/python3.10/site-packages/ray/rllib/algorithms/algorithm.py:1688: DeprecationWarning: invalid escape sequence '\|'
  """Adds a new policy to this Algorithm.


KeyboardInterrupt: 